### Segmentação Semântica com U-NET

A segmentação semântica e a de instâncias tem sido usada amplamente no setor de Saúde durante a pandemia de COVID-19 para ajudar os médicos a distinguir, entre as imagens de raios-X de vários pulmões, quais exibem pneumonia normal e quais exibem infecção por Coronavirus.

### Importação das bibliotecas

In [1]:
import os
import numpy as np
import cv2
from tqdm import tqdm # Mostrar barra de progresso
import matplotlib.pyplot as plt
import tensorflow
import keras
import random
tensorflow.__version__

'2.18.0'

In [2]:
# Utilizado para ocultar alguns avisos do tensorflow nas células do notebook na execução que pode prejudicar um pouco a visualização
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

### Dataset

**Data Science Bowl de 2018** - https://www.kaggle.com/competitions/data-science-bowl-2018

Antes (com os exemplos de segmentação de instâncias) trabalhamos com máscaras em um arquivo json que armazena as posições x e y dos objetos. Com este dataset trabalharemos com uma forma alternativa para armazenar os segmentos, no caso, as máscaras também serão imagens.

In [3]:
os.environ['KAGGLE_USERNAME'] = 'wolfros'
os.environ['KAGGLE_KEY'] = 'c1df133274141a6dea216267d44c22db'

Antes de fazer o download, é necessário baixar o kaggle, criar uma chave de api no kaggle e aceitar os termos da competição.

In [4]:
#!pip install kaggle

Baixando o dataset.

In [5]:
#!kaggle competitions download -c data-science-bowl-2018

Caso não exista pasta 'data', cria uma e descompacta os dados dentro desta pasta.

In [6]:
#!mkdir data && tar -xf data-science-bowl-2018.zip -C data

Verifica em qual diretório está.

In [7]:
#%pwd

Entra na pasta data, cria uma nova pasta 'train' caso não exista e descompata nela os dados de treino.

In [8]:
#%cd data
#!mkdir train && tar -xf stage1_train.zip -C train

Cria uma pasta 'test' caso não exista e descompacta nela os dados de teste.

In [9]:
#!mkdir test && tar -xf stage1_test.zip -C test

Volta para o diretório raiz.

In [10]:
#%cd ..

### Carregamento das imagens

In [18]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

SEED = 42 # Definindo seed para reprodutibilidade de resultados
np.random.seed = SEED
random.seed = SEED

PATH_TRAIN = 'data/train/'
PATH_TEST = 'data/test/'

Os nomes das pastas em cada pasta (train e test) são os mesmos nomes dos arquivos.

In [23]:
# Retorna o próximo valor de uma lista
# 1° elemento retorna o path
# 2° elemento retorna o array de itens (no caso nome das pastas que contem as imagens)
train_ids = next(os.walk(PATH_TRAIN))[1] # Estamos pegando o 2° elemento

# Visualizando os 5 primeiros ids das imagens de treino
for id in range(0,5):
    print(train_ids[id])

00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552
003cee89357d9fe13516167fd67b609a164651b21934585648c740d2c3d86dc1
00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b0740d00530220832c6d3e
0121d6759c5adb290c8e828fc882f37dfaf3663ec885c663859948c154a443ed
01d44a26f6680c42ba94c9bc6339228579a95d0e2695b149b7cc0c9592b21baf


In [24]:
# Fazemos a mesma coisa para os dados de teste
test_ids = next(os.walk(PATH_TEST))[1]

In [26]:
print(f'{len(train_ids)} para treinamento \n {len(test_ids)} para teste.')

670 para treinamento 
 65 para teste.


A máscara da imagem é composta somente pelo pixel 0 (zero) que representa o fundo e pelo pixel 1 (no caso, será 255 após ser multiplicado por 255) que representa onde está o objeto, na cor branca.

In [38]:
# Criando um array composto de zeros onde:
#                       qtd de imagens   altura     largura    canais (rgb)
dataset_img = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

# Fazemos a mesma coisa para as máscaras, porém com apenas 1 canal e o tipo de dados booleano (0 e 1)
dataset_mask = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

dataset_img.shape, dataset_mask.shape # A única diferença é o número de canais

((670, 256, 256, 3), (670, 256, 256, 1))